In [5]:
!nvidia-smi

Mon Nov  3 12:12:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 581.57                 Driver Version: 581.57         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8              3W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
#import matplotlib.pyplot as plt

In [6]:
# ===== 設定亂數種子，可以固定結果 =====
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# TODO - 自行選擇亂數種子
set_seed(20251024)

In [35]:
# ===== 載入訓練資料集 =====
train_df = pd.read_csv("train.csv")
# 預覽資料集
row_index = 0
row = train_df.iloc[row_index]
for col, val in row.items():
    if isinstance(val, str) and len(val) > 100:
        print(f"{col:25}: {val[:100]}...")   # truncate long text
    else:
        print(f"{col:25}: {val}")
# 數值與文字特徵
numeric_features = [c for c in train_df.select_dtypes(include=['number']).columns if c != 'metacritic_score']
print("All available numeric features :", numeric_features)
print("All available text features :", train_df.select_dtypes(include=['object']).columns.tolist())
print(train_df.shape)
print(len(numeric_features))

appid                    : 249130
name                     : LEGO® Marvel™ Super Heroes
release_date             : 1382400000
required_age             : 0
price                    : 19.99
dlc_count                : 2
detailed_description     : LEGO® Marvel™ Super Heroes features an original story crossing the entire Marvel Universe. Players t...
about_the_game           : LEGO® Marvel™ Super Heroes features an original story crossing the entire Marvel Universe. Players t...
short_description        : LEGO® Marvel™ Super Heroes features an original story crossing the entire Marvel Universe. Players t...
reviews                  : nan
windows                  : 1
mac                      : 0
linux                    : 0
achievements             : 45
recommendations          : 18110
positive                 : 24279
negative                 : 1196
estimated_owners         : 1500000
average_playtime_forever : 0
average_playtime_2weeks  : 0
median_playtime_forever  : 0
median_playtime_2weeks

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [64]:
def force_convert_and_validate(df, target_col='metacritic_score'):
    """
    强制转换所有列为数值，并验证转换成功率
    """
    results = {}
    usable_numeric_cols = []
    
    feature_cols = [col for col in df.columns if col != target_col]
    
    for col in feature_cols:
        # print(f"\n处理列: {col}")
        # print(f"  原始数据类型: {df[col].dtype}")
        
        # 强制转换为数值，无法转换的设为NaN
        converted = pd.to_numeric(df[col], errors='coerce')
        
        # 统计转换结果
        total_count = len(converted)
        numeric_count = converted.notna().sum()
        nan_count = converted.isna().sum()
        conversion_rate = numeric_count / total_count
        
        # 找出无法转换的值
        non_numeric_values = df[col][converted.isna() & df[col].notna()].unique()
        
        results[col] = {
            'conversion_rate': conversion_rate,
            'numeric_count': numeric_count,
            'nan_count': nan_count,
            'non_numeric_samples': non_numeric_values[:5] if len(non_numeric_values) > 0 else [],
            'converted_series': converted
        }
        
        #print(f"  转换率: {conversion_rate:.1%}")
        #print(f"  数值数据: {numeric_count}, 无法转换: {nan_count}")
        
        # if non_numeric_values.any():
        #     print(f"  非数值示例: {list(non_numeric_values[:3])}")
        
        # 如果转换率高于阈值，认为该列可用
        if conversion_rate > 0.8:  # 80%的数据能转换为数值
            usable_numeric_cols.append(col)
        #     print(f"  ✅ 可用作数值特征")
        # else:
        #     print(f"  ❌ 不适合作为数值特征")
    
    return usable_numeric_cols, results

# 强制转换并验证
usable_cols, conversion_results = force_convert_and_validate(train_df)
print(f"\n最终可用的数值列: {usable_cols}")


最终可用的数值列: ['appid', 'release_date', 'required_age', 'price', 'dlc_count', 'windows', 'mac', 'linux', 'achievements', 'recommendations', 'positive', 'negative', 'estimated_owners', 'average_playtime_forever', 'average_playtime_2weeks', 'median_playtime_forever', 'median_playtime_2weeks', 'discount', 'peak_ccu', 'pct_pos_total', 'num_reviews_total', 'pct_pos_recent', 'num_reviews_recent', 'has_website', 'has_support_url', 'has_support_email', 'support_english', 'support_japanese', 'support_spanish', 'support_chinese', 'support_french', 'is_top_20_developers', 'is_top_20_publishers', 'is_action', 'is_adventure', 'is_rpg', 'is_f2p', 'is_strategy', 'num_screenshots', 'num_movies']


In [55]:
target_column = 'metacritic_score'
#numeric_features = strict_numeric_cols(train_df)
len(strict_numeric_cols)

40

In [65]:
def improved_preprocess_data(df, target_col='metacritic_score'):
    """
    改进的预处理方法 - 使用封顶而不是移除样本
    """
    df_processed = df.copy()
    
    # 1. 处理缺失值（保持不变）
    print("处理缺失值...")
    numeric_cols,_ =  force_convert_and_validate(df_processed)
    
    for col in numeric_cols:
        if col in df_processed.columns and df_processed[col].isnull().sum() > 0:
            df_processed[col].fillna(df_processed[col].median(), inplace=True)
    
    # 2. 改进的异常值处理 - 使用封顶而不是移除
    print("改进的异常值处理（封顶方法）...")
    
    def cap_outliers_iqr(series):
        """使用IQR方法识别异常值并封顶，而不是移除"""
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # 封顶而不是移除
        capped_series = series.copy()
        capped_series = np.where(capped_series < lower_bound, lower_bound, capped_series)
        capped_series = np.where(capped_series > upper_bound, upper_bound, capped_series)
        
        return capped_series, lower_bound, upper_bound
    
    # 定义需要处理的数值特征
    numeric_features_to_cap = ['price', 'achievements', 'recommendations', 'positive', 'negative', 
                              'average_playtime_forever', 'peak_ccu', 'num_reviews_total']
    
    capping_info = {}
    original_shape = df_processed.shape[0]
    
    for col in numeric_features_to_cap:
        if col in df_processed.columns:
            original_outliers = len(df_processed[(df_processed[col] < df_processed[col].quantile(0.25) - 1.5*(df_processed[col].quantile(0.75)-df_processed[col].quantile(0.25))) | 
                                               (df_processed[col] > df_processed[col].quantile(0.75) + 1.5*(df_processed[col].quantile(0.75)-df_processed[col].quantile(0.25)))])
            
            df_processed[col], lower_bound, upper_bound = cap_outliers_iqr(df_processed[col])
            
            new_outliers = len(df_processed[(df_processed[col] == lower_bound) | (df_processed[col] == upper_bound)])
            
            capping_info[col] = {
                'original_outliers': original_outliers,
                'capped_samples': new_outliers,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound
            }
    
    print(f"样本数量保持不变: {original_shape}")
    
    # 3. 特征工程（保持不变）
    print("特征工程...")
    
    # 创建新特征
    if all(col in df_processed.columns for col in ['positive', 'negative']):
        df_processed['total_reviews'] = df_processed['positive'] + df_processed['negative']
        df_processed['positive_ratio'] = df_processed['positive'] / (df_processed['total_reviews'] + 1e-8)
    
    if all(col in df_processed.columns for col in ['average_playtime_forever', 'median_playtime_forever']):
        df_processed['playtime_variability'] = df_processed['average_playtime_forever'] - df_processed['median_playtime_forever']
    
    # 平台支持数量
    platform_cols = ['windows', 'mac', 'linux']
    if all(col in df_processed.columns for col in platform_cols):
        df_processed['platform_count'] = df_processed[platform_cols].sum(axis=1)
    
    print("\n=== 异常值封顶统计 ===")
    for col, info in capping_info.items():
        print(f"{col:25s}: {info['original_outliers']:3d} 个异常值被封顶到 [{info['lower_bound']:8.2f}, {info['upper_bound']:8.2f}]")
    
    return df_processed

train_processed_improved = improved_preprocess_data(train_df)
print(f"\n改进方法处理后的数据形状: {train_processed_improved.shape}")

处理缺失值...
改进的异常值处理（封顶方法）...
样本数量保持不变: 3200
特征工程...

=== 异常值封顶统计 ===
price                    : 308 个异常值被封顶到 [   -5.32,    35.18]
achievements             : 145 个异常值被封顶到 [  -28.00,    92.00]
recommendations          : 466 个异常值被封顶到 [-4855.25,  8518.75]
positive                 : 466 个异常值被封顶到 [-5566.88,  9744.12]
negative                 : 441 个异常值被封顶到 [ -865.25,  1556.75]
average_playtime_forever : 413 个异常值被封顶到 [ -402.00,   670.00]
peak_ccu                 : 568 个异常值被封顶到 [  -57.00,    95.00]
num_reviews_total        : 477 个异常值被封顶到 [-5206.25,  9137.75]

改进方法处理后的数据形状: (3200, 50)


In [70]:
target_column = 'metacritic_score'
#feature_columns = [col for col in train_processed_improved.columns if col != target_column]
X = train_df[usable_cols]  # 特征矩阵
y = train_df[target_column]    # 目标向量

In [73]:
def improved_feature_selection(X, y, feature_names, alpha=0.01):
    """
    改进的特征选择 - 包含标准化和多种重要性评估
    """
    from sklearn.preprocessing import StandardScaler
    from sklearn.inspection import permutation_importance
    
    # 分割数据集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # 1. 特征标准化 - 关键步骤！
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 定义模型
    models = {
        'Linear Regression': LinearRegression(),
        'Lasso Regression': Lasso(alpha=alpha, random_state=42),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\n=== {name} ===")
        
        # 训练模型
        if name == 'Random Forest':
            # 随机森林不需要标准化
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            feature_importance = model.feature_importances_
        else:
            # 线性模型使用标准化数据
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            if hasattr(model, 'coef_'):
                feature_importance = model.coef_
            else:
                feature_importance = np.zeros(X.shape[1])
        
        # 计算性能指标
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # 2. 计算排列重要性
        perm_importance = permutation_importance(
            model, X_test_scaled if name != 'Random Forest' else X_test, 
            y_test, n_repeats=10, random_state=42
        )
        
        # 3. 计算与目标的相关性
        correlations = []
        for i, feature in enumerate(feature_names):
            if feature in X.columns:
                corr = np.corrcoef(X[feature], y)[0, 1]
                correlations.append(corr)
        
        # 创建综合特征重要性DataFrame
        feature_importance_df = pd.DataFrame({
            'Feature': feature_names,
            'Standardized_Coefficient': feature_importance if name != 'Random Forest' else feature_importance,
            'Permutation_Importance': perm_importance.importances_mean,
            'Correlation_with_Target': correlations,
            'Absolute_Impact': np.abs(feature_importance) if name != 'Random Forest' else feature_importance
        })
        
        # 4. 计算综合得分
        if name != 'Random Forest':
            # 对于线性模型，使用标准化系数的绝对值
            feature_importance_df['Composite_Score'] = (
                np.abs(feature_importance_df['Standardized_Coefficient']) * 0.4 +
                feature_importance_df['Permutation_Importance'] * 0.3 +
                np.abs(feature_importance_df['Correlation_with_Target']) * 0.3
            )
        else:
            # 对于随机森林，直接使用特征重要性
            feature_importance_df['Composite_Score'] = feature_importance_df['Absolute_Impact']
        
        # 按综合得分排序
        feature_importance_df = feature_importance_df.sort_values('Composite_Score', ascending=False)
        
        results[name] = {
            'model': model,
            'feature_importance': feature_importance_df,
            'mse': mse,
            'r2': r2
        }
        
        print(f"MSE: {mse:.4f}, R²: {r2:.4f}")
        print("\n综合特征重要性排名:")
        print(feature_importance_df.head(15).round(4))
        
        # 5. 提供解释指南
        print(f"\n重要性解释指南:")
        print(f"标准化系数: 每标准差变化对目标的影响")
        print(f"排列重要性: 随机打乱特征后性能下降程度") 
        print(f"相关性: 与目标的线性关系强度")
        print(f"综合得分: 加权总体重要性")
    
    return results, scaler

In [75]:
# 新增：特征重要性分类函数
def classify_feature_importance(feature_importance_df, model_name):
    """
    根据综合得分对特征重要性进行分类
    """
    scores = feature_importance_df['Composite_Score'].values
    
    # 动态阈值
    high_threshold = np.percentile(scores, 75)  # 前25%
    medium_threshold = np.percentile(scores, 50)  # 前50%
    low_threshold = np.percentile(scores, 25)  # 前75%
    
    print(f"\n=== {model_name} 特征重要性分类 ===")
    print(f"阈值 - 高: >{high_threshold:.4f}, 中: {medium_threshold:.4f}-{high_threshold:.4f}, 低: <{medium_threshold:.4f}")
    
    high_impact = feature_importance_df[feature_importance_df['Composite_Score'] > high_threshold]
    medium_impact = feature_importance_df[
        (feature_importance_df['Composite_Score'] > medium_threshold) & 
        (feature_importance_df['Composite_Score'] <= high_threshold)
    ]
    low_impact = feature_importance_df[feature_importance_df['Composite_Score'] <= medium_threshold]
    
    print(f"\n高影响力特征 ({len(high_impact)}个):")
    for _, row in high_impact.iterrows():
        print(f"  {row['Feature']:25s} | 得分: {row['Composite_Score']:.4f} | 相关性: {row['Correlation_with_Target']:.3f}")
    
    return high_impact, medium_impact, low_impact

# 使用改进的方法
results, scaler = improved_feature_selection(X, y, usable_cols)

# 对每个模型进行分类
for model_name in results.keys():
    feature_importance_df = results[model_name]['feature_importance']
    high, medium, low = classify_feature_importance(feature_importance_df, model_name)


=== Linear Regression ===


c:\enlist\LLM\Finetuning\tuning_env\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\enlist\LLM\Finetuning\tuning_env\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


MSE: 13.5698, R²: 0.6736

综合特征重要性排名:
                     Feature  Standardized_Coefficient  \
19             pct_pos_total                    3.4720   
21            pct_pos_recent                    2.1739   
27          support_japanese                    0.6358   
15   median_playtime_forever                    0.6618   
25         has_support_email                   -0.6026   
9            recommendations                    0.5679   
10                  positive                   -0.4550   
3                      price                    0.5164   
12          estimated_owners                    0.4553   
29           support_chinese                   -0.4574   
13  average_playtime_forever                   -0.3000   
0                      appid                    0.2954   
20         num_reviews_total                    0.1817   
6                        mac                    0.3024   
1               release_date                   -0.1974   

    Permutation_Importance  Correl

c:\enlist\LLM\Finetuning\tuning_env\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\enlist\LLM\Finetuning\tuning_env\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


MSE: 13.8819, R²: 0.6661

综合特征重要性排名:
                     Feature  Standardized_Coefficient  \
21            pct_pos_recent                    0.3945   
19             pct_pos_total                    0.2760   
3                      price                    0.0245   
8               achievements                    0.0241   
18                  peak_ccu                    0.0238   
0                      appid                    0.0228   
1               release_date                    0.0225   
10                  positive                    0.0207   
38           num_screenshots                    0.0203   
9            recommendations                    0.0164   
11                  negative                    0.0159   
22        num_reviews_recent                    0.0152   
20         num_reviews_total                    0.0146   
13  average_playtime_forever                    0.0131   
15   median_playtime_forever                    0.0126   

    Permutation_Importance  Correl

c:\enlist\LLM\Finetuning\tuning_env\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\enlist\LLM\Finetuning\tuning_env\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [77]:
# TODO(Boss baseline) - Feature Selection: 選擇適合的features進行訓練，可選features請參見csv檔欄位
# To check all available numeric features, uncomment the line below :
# print("All availbale numeric features :", train_df.select_dtypes(include=['number']).columns.tolist())
numeric_features = ['pct_pos_recent', 'pct_pos_total', 'achievements','price','peak_ccu','appid','release_date','positive','num_screenshots','recommendations']

X_numeric = train_df[numeric_features].fillna(0).values
y = train_df['metacritic_score'].fillna(0).values.reshape(-1, 1)


# 預設只使用數值特徵
X = X_numeric

# TODO(Boss baseline) - Feature Selection: 使用文字欄位的Embedding捕捉特徵
"""
# 如果想要使用embedding，可以取消以下程式碼註解
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# 要進行embedding的文字欄位
text_columns = ['reviews','short_description']

embedder = SentenceTransformer('all-mpnet-base-v2')

def extract_embeddings(df, col):
    print(f"Embedding column: {col}")
    texts = df[col].fillna("").astype(str).tolist()
    emb = embedder.encode(texts, batch_size=64, show_progress_bar=True, convert_to_numpy=True)
    return emb

def reduce_dim_with_pca(embeddings, n_components=64):
    pca = PCA(n_components=n_components)
    reduced = pca.fit_transform(embeddings)
    return reduced, pca

# 建立embeddings並用PCA降維
embeddings_reduced = []
pca_models = {}
for col in text_columns:
    emb = extract_embeddings(train_df, col)
    emb_reduced, pca_model = reduce_dim_with_pca(emb, n_components=100)
    embeddings_reduced.append(emb_reduced)
    pca_models[col] = pca_model

X_text_reduced = np.concatenate(embeddings_reduced, axis=1)

# 將數值特徵與文字特徵結合
X = np.concatenate([X_numeric, X_text_reduced], axis=1)
"""

# ===== 對資料進行標準化 =====
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y)

# ===== 切分 train/valid set: 可以自行調整訓練集與驗證集的比例 =====
X_train, X_dev, y_train, y_dev = train_test_split(X_scaled, y_scaled, test_size=0.25, random_state=42)

In [78]:
class SteamDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.float32)
    def __len__(self):
        return len(self.features)
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [79]:
# TODO(Strong baseline) - 模型架構: 自行調整模型架構以提高模型能力
class LinearModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, 1)

    def forward(self, x):
        return self.fc(x)

class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims=(128, 64), dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]), # 第一層：input_dim -> 128，後接ReLU，好處是能夠引入非線性，讓模型學習更複雜的模式
            nn.ReLU(),
            nn.Dropout(dropout), # 設置Dropout能夠防止overfitting
            nn.Linear(hidden_dims[0], hidden_dims[1]), # 中間層：128 -> 64，接ReLU
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dims[1], 1), # 輸出層：64 -> 1
        )

    def forward(self, x):
        return self.net(x)

In [82]:
def training(model, train_loader, dev_loader, n_epochs, target_scaler, optimizer):
    loss_fn = nn.MSELoss()
    train_loss_history = []
    val_rmse_history = []
    device = next(model.parameters()).device

    for epoch in range(n_epochs):
        # === 训练阶段 ===
        model.train()
        total_loss = 0

        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            preds = model(x_batch)
            loss = loss_fn(preds, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        train_loss_history.append(avg_loss)

        # === 验证阶段 ===
        model.eval()
        all_preds, all_targets = [], []

        with torch.no_grad():
            for x_batch, y_batch in dev_loader:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                preds = model(x_batch)
                all_preds.append(preds.cpu())
                all_targets.append(y_batch.cpu())

        # 反标准化和计算RMSE
        all_preds = torch.cat(all_preds).numpy()
        all_targets = torch.cat(all_targets).numpy()

        y_pred_inv = target_scaler.inverse_transform(all_preds.reshape(-1, 1)).flatten()
        y_true_inv = target_scaler.inverse_transform(all_targets.reshape(-1, 1)).flatten()

        rmse = np.sqrt(mean_squared_error(y_true_inv, y_pred_inv))
        val_rmse_history.append(rmse)

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}: Train Loss={avg_loss:.4f}, Val RMSE={rmse:.4f}")
    result = {
        'final_train_loss': train_loss_history[-1],
        'final_val_rmse': val_rmse_history[-1],
        'best_val_rmse': min(val_rmse_history),
        'best_epoch': np.argmin(val_rmse_history) + 1,
    }
    #print(result)
    return result

In [ ]:
# TODO(Medium baseline) - Training Hyperparameters: 自行調整訓練超參數來改善模型表現
n_epochs = 3000          # 訓練回合數
learning_rate = 1e-3     # 學習率
batch_size = 128         # 每次訓練取多少樣本
weight_decay = 1e-5         # L2 regularization強度，緩解overfitting。Useful Reference:https://medium.com/analytics-vidhya/deep-learning-basics-weight-decay-3c68eb4344e9
opt_name = 'Adam'

train_set = SteamDataset(X_train, y_train)
dev_set = SteamDataset(X_dev, y_dev)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False)
dev_loader = DataLoader(dev_set, batch_size=batch_size, shuffle=False)

target_scaler = StandardScaler()
target_scaler.fit(y_train.reshape(-1, 1))

model = MLPModel(input_dim=X_train.shape[1]).to(device)

            # 选择优化器
if opt_name == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
else:  # SGD
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay, momentum=0.9)

# 训练模型
detailed_results = training(
    model=model,
    train_loader=train_loader,
    dev_loader=dev_loader,
    n_epochs=n_epochs,
    target_scaler=target_scaler, # Moved target_scaler here
    optimizer=optimizer
)
print(detailed_results)

Epoch 50: Train Loss=0.3201, Val RMSE=0.5709
Epoch 100: Train Loss=0.3041, Val RMSE=0.5709
Epoch 150: Train Loss=0.2922, Val RMSE=0.5763
Epoch 200: Train Loss=0.2927, Val RMSE=0.5791
Epoch 250: Train Loss=0.2903, Val RMSE=0.5833
Epoch 300: Train Loss=0.2825, Val RMSE=0.5817
Epoch 350: Train Loss=0.2751, Val RMSE=0.5831
Epoch 400: Train Loss=0.2818, Val RMSE=0.5813
Epoch 450: Train Loss=0.2671, Val RMSE=0.5825
Epoch 500: Train Loss=0.2655, Val RMSE=0.5856
Epoch 550: Train Loss=0.2666, Val RMSE=0.5895
Epoch 600: Train Loss=0.2576, Val RMSE=0.5830
Epoch 650: Train Loss=0.2559, Val RMSE=0.5839
Epoch 700: Train Loss=0.2530, Val RMSE=0.5903
Epoch 750: Train Loss=0.2595, Val RMSE=0.5914
Epoch 800: Train Loss=0.2566, Val RMSE=0.5888
Epoch 850: Train Loss=0.2515, Val RMSE=0.5945
Epoch 900: Train Loss=0.2430, Val RMSE=0.5883
Epoch 950: Train Loss=0.2524, Val RMSE=0.5910
Epoch 1000: Train Loss=0.2402, Val RMSE=0.5939
Epoch 1050: Train Loss=0.2461, Val RMSE=0.5905
Epoch 1100: Train Loss=0.2386, Va

In [24]:
import itertools
import torch
import torch.nn as nn
from tqdm import tqdm
import pandas as pd
import json
from datetime import datetime

def hyperparameter_search(model_class, input_dim, train_dataset, dev_dataset, target_scaler):
    # 定义超参数网格
    n_epochs_list = [1000, 2000, 3000, 4000, 5000] #
    learning_rate_list = [1e-2, 1e-3, 1e-4]
    batch_size_list = [64, 128] # 8, 16, 32, 
    weight_decay_list = [1e-3, 1e-4, 1e-5]
    optimizer_list = ['Adam', 'SGD']

    # 存储所有结果
    results = []
    best_rmse = float('inf')
    best_params = None

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # 生成所有组合
    total_combinations = len(n_epochs_list) * len(learning_rate_list) * len(batch_size_list) * len(weight_decay_list) * len(optimizer_list)

    print(f"开始超参数搜索，总共 {total_combinations} 种组合")

    # 遍历所有组合
    for i, (n_epochs, lr, batch_size, wd, opt_name) in enumerate(
        itertools.product(n_epochs_list, learning_rate_list, batch_size_list, weight_decay_list, optimizer_list)
    ):
        print(f"\n{'='*60}")
        print(f"进度: {i+1}/{total_combinations}")
        print(f"参数组合: n_epochs={n_epochs}, lr={lr}, batch_size={batch_size}, weight_decay={wd}, optimizer={opt_name}")

        try:
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
            dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

            # 创建新模型实例
            model = model_class(input_dim=input_dim).to(device)

            # 选择优化器
            if opt_name == 'Adam':
                optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
            else:  # SGD
                optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)

            # 训练模型
            detailed_results = training(
                model=model,
                train_loader=train_loader,
                dev_loader=dev_loader,
                n_epochs=n_epochs,
                target_scaler=target_scaler, # Moved target_scaler here
                optimizer=optimizer
            )

            # 获取最佳结果
            final_train_loss = detailed_results['final_train_loss'] # Corrected access
            final_val_rmse = detailed_results['final_val_rmse'] # Corrected access
            best_val_rmse = detailed_results['best_val_rmse'] # Corrected access

            #记录结果
            result = {
                'n_epochs': n_epochs,
                'learning_rate': lr,
                'batch_size': batch_size,
                'weight_decay': wd,
                'optimizer': opt_name,
                'final_train_loss': final_train_loss,
                'final_val_rmse': final_val_rmse,
                'best_val_rmse': best_val_rmse,
                'best_epoch': detailed_results['best_epoch'],
            }

            results.append(result)
            print(f"结果: 最终训练损失={final_train_loss:.4f}, 最终验证RMSE={final_val_rmse:.4f}, 最佳验证RMSE={best_val_rmse:.4f}")
            # 更新最佳参数
            if best_val_rmse < best_rmse:
                best_rmse = best_val_rmse
                print(f"🎯 新的最佳结果! RMSE: {best_rmse:.4f}")

                # 保存最佳模型
                # torch.save(model.state_dict(), f'best_model_rmse_{best_rmse:.4f}.pth')
        except Exception as e:
            print(f"❌ 组合训练失败: {e}")
            continue
    # 保存最终结果
    save_final_results(results, best_params)
    return results, best_params

def save_final_results(results, best_params):
    """保存最终结果"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 完整的DataFrame
    df = pd.DataFrame([{
        'n_epochs': r['n_epochs'],
        'learning_rate': r['learning_rate'],
        'batch_size': r['batch_size'],
        'weight_decay': r['weight_decay'],
        'optimizer': r['optimizer'],
        'final_train_loss': r['final_train_loss'],
        'final_val_rmse': r['final_val_rmse'],
        'best_val_rmse': r['best_val_rmse'],
        'best_epoch': r['best_epoch']
    } for r in results])

    # 按最佳RMSE排序
    df_sorted = df.sort_values('best_val_rmse')

    # 保存结果
    #/content/drive/MyDrive/Colab Notebooks/GenAICourse/Copy of Gen-AI-ML-Hw1.ipynb
    df_sorted.to_csv(f'hyperparameter_results_{timestamp}.csv', index=False)

    print(f"\n{'='*60}")
    print("🎉 超参数搜索完成!")
    if best_params: # Check if best_params is not None
        print(f"最佳参数组合:")
        for key, value in best_params.items():
            if key not in ['train_loss_history', 'val_rmse_history']:
                print(f"  {key}: {value}")
        print(f"最佳验证RMSE: {best_params['best_val_rmse']:.4f}")
    else:
        print("未能找到最佳参数组合。")
    print(f"结果已保存到: hyperparameter_results_{timestamp}.csv")

In [25]:
train_set = SteamDataset(X_train, y_train)
dev_set = SteamDataset(X_dev, y_dev)

target_scaler = StandardScaler()
target_scaler.fit(y_train.reshape(-1, 1))
# def hyperparameter_search(model_class, input_dim, train_dataset, dev_dataset, target_scaler):
results, best_params = hyperparameter_search(
        model_class=MLPModel,  # 替换为您的模型类
        input_dim=X_train.shape[1],
        train_dataset=train_set,
        dev_dataset=dev_set,
        target_scaler=target_scaler
)

开始超参数搜索，总共 180 种组合

进度: 1/180
参数组合: n_epochs=1000, lr=0.01, batch_size=64, weight_decay=0.001, optimizer=Adam
结果: 最终训练损失=0.6166, 最终验证RMSE=0.7758, 最佳验证RMSE=0.7490
🎯 新的最佳结果! RMSE: 0.7490

进度: 2/180
参数组合: n_epochs=1000, lr=0.01, batch_size=64, weight_decay=0.001, optimizer=SGD
结果: 最终训练损失=0.5916, 最终验证RMSE=0.7729, 最佳验证RMSE=0.7451
🎯 新的最佳结果! RMSE: 0.7451

进度: 3/180
参数组合: n_epochs=1000, lr=0.01, batch_size=64, weight_decay=0.0001, optimizer=Adam
结果: 最终训练损失=0.5378, 最终验证RMSE=0.7284, 最佳验证RMSE=0.7172
🎯 新的最佳结果! RMSE: 0.7172

进度: 4/180
参数组合: n_epochs=1000, lr=0.01, batch_size=64, weight_decay=0.0001, optimizer=SGD
结果: 最终训练损失=0.5642, 最终验证RMSE=0.7439, 最佳验证RMSE=0.7317

进度: 5/180
参数组合: n_epochs=1000, lr=0.01, batch_size=64, weight_decay=1e-05, optimizer=Adam
结果: 最终训练损失=0.5331, 最终验证RMSE=0.7193, 最佳验证RMSE=0.7129
🎯 新的最佳结果! RMSE: 0.7129

进度: 6/180
参数组合: n_epochs=1000, lr=0.01, batch_size=64, weight_decay=1e-05, optimizer=SGD
结果: 最终训练损失=0.5392, 最终验证RMSE=0.7320, 最佳验证RMSE=0.7245

进度: 7/180
参数组合: n_epochs=1000, 

In [20]:
# TODO(Medium baseline) - Training Hyperparameters: 自行調整訓練超參數來改善模型表現
n_epochs = 1000          # 訓練回合數
learning_rate = 1e-2     # 學習率
batch_size = 8         # 每次訓練取多少樣本
weight_decay = 0.001         # L2 regularization強度，緩解overfitting。Useful Reference:https://medium.com/analytics-vidhya/deep-learning-basics-weight-decay-3c68eb4344e9
shuffle_data = True      # 是否在每個 epoch 打亂資料訓練

In [21]:
# TODO(Medium baseline) - Training Hyperparameters: 自行調整訓練超參數來改善模型表現
n_epochs = 1000          # 訓練回合數
learning_rate = 1e-2     # 學習率
batch_size = 8         # 每次訓練取多少樣本
weight_decay = 0.001         # L2 regularization強度，緩解overfitting。Useful Reference:https://medium.com/analytics-vidhya/deep-learning-basics-weight-decay-3c68eb4344e9
shuffle_data = True      # 是否在每個 epoch 打亂資料訓練

# ===== 用DataLoader讀取資料集 =====
train_set = SteamDataset(X_train, y_train)
dev_set = SteamDataset(X_dev, y_dev)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=shuffle_data)
dev_loader = DataLoader(dev_set, batch_size=batch_size, shuffle=False)

# ===== Model / Optimizer / Loss =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLPModel(input_dim=X_train.shape[1]).to(device)  # TODO(Strong baseline) - 也可以換成 MLPModel LinearModel
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
loss_fn = nn.MSELoss()
# ===== 訓練迴圈 =====
train_loss_history = []
val_rmse_history = []
for epoch in tqdm(range(n_epochs), desc="Training Progress"):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        preds = model(x_batch)
        loss = loss_fn(preds, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    train_loss_history.append(avg_loss)
    # 驗證
    model.eval()
    preds_list, y_true_list = [], []
    with torch.no_grad():
        for x_batch, y_batch in dev_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            preds = model(x_batch)
            preds_list.append(preds.cpu().numpy())  # move back to CPU for numpy
            y_true_list.append(y_batch.cpu().numpy())
    y_pred_raw = np.vstack(preds_list)
    y_true_raw = np.vstack(y_true_list)

    y_pred_inv = target_scaler.inverse_transform(y_pred_raw)
    y_true_inv = target_scaler.inverse_transform(y_true_raw)

    rmse = np.sqrt(mean_squared_error(y_true_inv, y_pred_inv))
    val_rmse_history.append(rmse)

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}: Train Loss={avg_loss:.4f}, Val RMSE={rmse:.4f}")
# ===== Plot Curves =====
epochs = np.arange(1, n_epochs + 1)

Training Progress:   5%|▌         | 50/1000 [00:38<12:36,  1.26it/s]

Epoch 50: Train Loss=0.7005, Val RMSE=1.1521


Training Progress:  10%|█         | 100/1000 [01:17<11:34,  1.30it/s]

Epoch 100: Train Loss=0.8421, Val RMSE=1.0399


Training Progress:  15%|█▌        | 150/1000 [01:57<10:45,  1.32it/s]

...

Epoch 900: Train Loss=0.7380, Val RMSE=0.7835


Training Progress:  95%|█████████▌| 950/1000 [12:33<00:39,  1.27it/s]

Epoch 950: Train Loss=0.7553, Val RMSE=0.7983


Training Progress: 100%|██████████| 1000/1000 [13:13<00:00,  1.26it/s]

Epoch 1000: Train Loss=0.6827, Val RMSE=0.8189
